#Install Library dan Packages

In [ ]:
!pip install deep-translator

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.1 MB/s eta 0:00:00


In [ ]:
!pip install nltk

In [ ]:
import nltk
nltk.download('vader_lexicon')
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import pandas as pd
import re
import numpy as np
import nltk
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from deep_translator import GoogleTranslator

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Text Preprocessing

In [ ]:
# read csv
df = pd.read_csv('drive/MyDrive/TA/Codingan TA/analisis sentimen /dataset/dataset-capres2024.csv')
df.head()


,Unnamed: 0,created_at,id_str,full_text,username,tweet_url
0,0,Sat Dec 30 23:53:23 +0000 2023,1741246132981059943,“Desak Anies “ Bentuk Pemimpin Sejati Bersama...,syahrizal3183,https://twitter.com/syahrizal3183/status/17412...
1,1,Sat Dec 30 23:10:38 +0000 2023,1741235375174193538,Pak anies sekarang jadi father figure buat par...,YasminAliyah23,https://twitter.com/YasminAliyah23/status/1741...
2,2,Sat Dec 30 23:18:16 +0000 2023,1741237294378697015,#anies bubble #AMINAjaDulu https://t.co/eQaMd...,ChelseaRiy6500,https://twitter.com/ChelseaRiy6500/status/1741...
3,3,Sat Dec 30 23:33:17 +0000 2023,1741241073920520306,Ngundhuh wohing pakerti Apa pun yang kita laku...,Ummat_Kebumen,https://twitter.com/Ummat_Kebumen/status/17412...
4,4,Sat Dec 30 23:03:08 +0000 2023,1741233487653577075,Dukungan kepad Pak @aniesbaswedan sungguh di l...,BulsTheRock,https://twitter.com/BulsTheRock/status/1741233...


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3986 entries, 0 to 3985
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  3986 non-null   int64 
 1   created_at  3986 non-null   object
 2   id_str      3986 non-null   int64 
 3   full_text   3986 non-null   object
 4   username    3986 non-null   object
 5   tweet_url   3986 non-null   object
dtypes: int64(2), object(4)
memory usage: 187.0+ KB


In [ ]:
df.shape

(3986, 6)

In [ ]:
# menghapus kolom yang tidak digunakan
df = df.drop(columns=[col for col in df.columns if col.startswith("Unnamed")])
df = df.drop(columns = ['id_str', 'tweet_url', 'created_at', 'username'])
df

,full_text
0,“Desak Anies “ Bentuk Pemimpin Sejati Bersama...
1,Pak anies sekarang jadi father figure buat par...
2,#anies bubble #AMINAjaDulu https://t.co/eQaMd...
3,Ngundhuh wohing pakerti Apa pun yang kita laku...
4,Dukungan kepad Pak @aniesbaswedan sungguh di l...
...,...
3981,"Bertandang ke Banten, Atikoh Sowan Abuya Muhta..."
3982,@RaniKancana selamat ngopi #GanjarMahfud https...
3983,"Di Garut, Ganjar Komitmen Beri Insentif Guru N..."
3984,@sosmedkeras sudah saat nya kita dukung #Ganja...


Text Cleaning

In [ ]:
def clean_tweet(text):
  # convert to lowercase(case folding)
  text = text.lower()
  # remove mention from tweet
  text = re.sub(r'@[A-Za-z0-9_]+', '', text)
  # remove hastag
  text = re.sub(r'#\w+', '', text)
  #remove retweet
  text = re.sub(r'RT[\s]+', '', text)
  # remove URLs
  text = re.sub(r'https?://\S+', '', text)
  # remove emojis
  text = re.sub(r'[^\w\s]', '', text)
  # remove newline characters
  text = re.sub(r'\n', ' ', text)
  # menghapus spesial charachters
  text = re.sub(r'[^A-Za-z0-9 ]', '', text)
  # remove extra whitespace
  text = re.sub(r'\s+', ' ', text).strip()
  # remove number
  text = re.sub(r'\d+', '',text)
  return text

In [ ]:
df['clean_tweet'] = df['full_text'].apply(clean_tweet)
df.head(10)

,full_text,clean_tweet
0,“Desak Anies “ Bentuk Pemimpin Sejati Bersama...,desak anies bentuk pemimpin sejati bersama rak...
1,Pak anies sekarang jadi father figure buat par...,pak anies sekarang jadi father figure buat par...
2,#anies bubble #AMINAjaDulu https://t.co/eQaMd...,bubble
3,Ngundhuh wohing pakerti Apa pun yang kita laku...,ngundhuh wohing pakerti apa pun yang kita laku...
4,Dukungan kepad Pak @aniesbaswedan sungguh di l...,dukungan kepad pak sungguh di luar ekspetasi a...
5,"Anies Bubble Trending X, Ini Cerita Anies Basw...",anies bubble trending x ini cerita anies baswe...
6,Jeritan Hati Driver Ojol !! #aniesbaswedan #j...,jeritan hati driver ojol
7,@OposisiCerdas Naik kemana..??😂 Rakyat cerdas...,naik kemana rakyat cerdas pilih
8,@deniss4lam @Naz_lira Semoga ikthiar yg kita l...,semoga ikthiar yg kita lakukan wabil khusus pa...
9,#aniesbaswedan #jokowi #prabowo #gibran #ganja...,


Normalisasi

In [ ]:
# Membaca file CSV yang berisi pasangan kata untuk normalisasi
normalizad_word = pd.read_csv('drive/MyDrive/TA/Codingan TA/analisis sentimen /dataset/normalisasi.csv', encoding='ISO-8859-1')

# Membangun dictionary dari pasangan kata
normalizad_word_dict = {}
for index, row in normalizad_word.iterrows():
    key = str(row[0]).strip()
    value = str(row[1]).strip()
    normalizad_word_dict[key] = value


# Fungsi untuk melakukan normalisasi teks menggunakan regex
def normalizad_term(document):
    for term, replacement in normalizad_word_dict.items():
        # Menggunakan regex untuk pencocokan kata yang tepat
        document = re.sub(rf'\b{re.escape(term)}\b', replacement, document)
    return document

# Melakukan normalisasi pada kolom 'tweet_tokens_WSW'
df['tweet_normalized'] = df['clean_tweet'].apply(normalizad_term)
df

,full_text,clean_tweet,tweet_normalized
0,“Desak Anies “ Bentuk Pemimpin Sejati Bersama...,desak anies bentuk pemimpin sejati bersama rak...,desak anies bentuk pemimpin sejati bersama rak...
1,Pak anies sekarang jadi father figure buat par...,pak anies sekarang jadi father figure buat par...,pak anies sekarang jadi father figure untuk pa...
2,#anies bubble #AMINAjaDulu https://t.co/eQaMd...,bubble,bubble
3,Ngundhuh wohing pakerti Apa pun yang kita laku...,ngundhuh wohing pakerti apa pun yang kita laku...,ngundhuh wohing pakerti apa pun yang kita laku...
4,Dukungan kepad Pak @aniesbaswedan sungguh di l...,dukungan kepad pak sungguh di luar ekspetasi a...,dukungan kepada pak sungguh di luar ekspetasi ...
...,...,...,...
3981,"Bertandang ke Banten, Atikoh Sowan Abuya Muhta...",bertandang ke banten atikoh sowan abuya muhtad...,bertandang ke banten atikoh sowan abuya muhtad...
3982,@RaniKancana selamat ngopi #GanjarMahfud https...,selamat ngopi,selamat ngopi
3983,"Di Garut, Ganjar Komitmen Beri Insentif Guru N...",di garut ganjar komitmen beri insentif guru ng...,di garut ganjar komitmen beri insentif guru ng...
3984,@sosmedkeras sudah saat nya kita dukung #Ganja...,sudah saat nya kita dukung untuk bereskan hal ...,sudah saat nya kita dukung untuk bereskan hal ...


In [ ]:
# save hasil text cleaning dan normalisasi ke dataset baru
df = df.rename(columns={'tweet_normalized': 'tweet_indonesia'})

new_df = df['tweet_indonesia']
new_df.head()

0    desak anies bentuk pemimpin sejati bersama rak...
1    pak anies sekarang jadi father figure untuk pa...
2                                               bubble
3    ngundhuh wohing pakerti apa pun yang kita laku...
4    dukungan kepada pak sungguh di luar ekspetasi ...
Name: tweet_indonesia, dtype: object

In [ ]:
# check data duplikat
new_df.duplicated().sum()

949

In [ ]:
# menghapus data duplikat
new_df = new_df.drop_duplicates()
new_df.duplicated().sum()

0

In [ ]:
# check missing value
new_df.isnull().sum()

0

In [ ]:
new_df.to_csv('drive/MyDrive/TA/Codingan TA/analisis sentimen /dataset/tweet_indo.csv', index=False)

In [ ]:
new_df.shape

(3037,)

Translate Dataset

In [ ]:
tweet = pd.read_csv('drive/MyDrive/TA/Codingan TA/analisis sentimen /dataset/tweet_indo.csv')
tweet

,tweet_indonesia
0,desak anies bentuk pemimpin sejati bersama rak...
1,pak anies sekarang jadi father figure untuk pa...
2,bubble
3,ngundhuh wohing pakerti apa pun yang kita laku...
4,dukungan kepada pak sungguh di luar ekspetasi ...
...,...
3032,bertandang ke banten atikoh sowan abuya muhtad...
3033,selamat ngopi
3034,di garut ganjar komitmen beri insentif guru ng...
3035,sudah saat nya kita dukung untuk bereskan hal ...


In [20]:
#inisialisasi penerjemah
translator = GoogleTranslator(source='id', target='en')

def translate_text(text):
  try:
    return translator.translate(text)
  except Exception as e:
    print(f'Error translationg text: {text}, error: {e}')
    return text

# menerjemahkan kolom 'full_text'
tweet['tweet_english'] = tweet['tweet_indonesia'].apply(translate_text)

# menyimpan hasil terjemahan ke file baru
tweet['tweet_english'].to_csv('drive/MyDrive/TA/Codingan TA/analisis sentimen /dataset/tweet_english.csv', index=False)

print("Translation complite and saved to 'tweet_english.csv'")

Error translationg text: nan, error: nan --> text must be a valid text with maximum 5000 character,otherwise it cannot be translated
Translation complite and saved to 'tweet_english.csv'


Pelabelan Data

In [21]:
tweet_eng = pd.read_csv('drive/MyDrive/TA/Codingan TA/analisis sentimen /dataset/tweet_english.csv')
tweet_eng.head()

,tweet_english
0,Anies urges to form a true leader with the peo...
1,"Mr. Anies is now a father figure for Gen Z, es..."
2,bubble
3,Downloading why whatever we do will produce co...
4,the support for sir is really beyond expectati...


In [22]:
tweet_eng.isnull().sum()

tweet_english    1
dtype: int64

In [23]:
tweet_eng = tweet_eng.dropna()
tweet_eng.isnull().sum()

tweet_english    0
dtype: int64

In [24]:
tweet_eng.duplicated().sum()

7

In [25]:
tweet_eng = tweet_eng.drop_duplicates()
tweet_eng.duplicated().sum()

0

In [26]:
tweet_eng.shape

(3029, 1)

In [27]:
# inisialisasi VADER
vader = SentimentIntensityAnalyzer()

def get_sentiment_label(text):
  score = vader.polarity_scores(text)
  compound = score['compound']
  if compound >=  0.05 :
    return 'positif'
  elif compound <= -0.05 :
    return 'negatif'
  else :
    return 'netral'


# menerapkan function untuk pelabelan
tweet_eng['labels'] = tweet_eng['tweet_english'].apply(get_sentiment_label)
tweet_eng.head(20)

,tweet_english,labels
0,Anies urges to form a true leader with the peo...,positif
1,"Mr. Anies is now a father figure for Gen Z, es...",positif
2,bubble,netral
3,Downloading why whatever we do will produce co...,netral
4,the support for sir is really beyond expectati...,positif
5,Anies Bubble Trending,netral
6,the heart-rending scream of the ojol driver,negatif
7,rise wherever the intelligent people choose,positif
8,I hope that the efforts we carry out are speci...,positif
10,it's really a cool top amp,positif


In [28]:
tweet_eng['labels'].value_counts()

labels
positif    1762
netral      770
negatif     497
Name: count, dtype: int64

In [29]:
tweet_eng.to_csv('drive/MyDrive/TA/Codingan TA/analisis sentimen /dataset/tweet_labels.csv', index=False)

Tokenisasi


In [30]:
tweet_label = pd.read_csv('drive/MyDrive/TA/Codingan TA/analisis sentimen /dataset/tweet_labels.csv')
tweet_label.head()

,tweet_english,labels
0,Anies urges to form a true leader with the peo...,positif
1,"Mr. Anies is now a father figure for Gen Z, es...",positif
2,bubble,netral
3,Downloading why whatever we do will produce co...,netral
4,the support for sir is really beyond expectati...,positif


In [31]:
def tokenize(text):
  return word_tokenize(text)

In [32]:
tweet_label['tokenize_tweet'] = tweet_label['tweet_english'].apply(tokenize)

tweet_label = tweet_label[['tweet_english', 'tokenize_tweet', 'labels']]
tweet_label

,tweet_english,tokenize_tweet,labels
0,Anies urges to form a true leader with the peo...,"[Anies, urges, to, form, a, true, leader, with...",positif
1,"Mr. Anies is now a father figure for Gen Z, es...","[Mr., Anies, is, now, a, father, figure, for, ...",positif
2,bubble,[bubble],netral
3,Downloading why whatever we do will produce co...,"[Downloading, why, whatever, we, do, will, pro...",netral
4,the support for sir is really beyond expectati...,"[the, support, for, sir, is, really, beyond, e...",positif
...,...,...,...
3024,"Visiting Banten, Atikoh visited Abuya Muhtadi ...","[Visiting, Banten, ,, Atikoh, visited, Abuya, ...",negatif
3025,Happy coffee,"[Happy, coffee]",positif
3026,in Garut reward commitment to provide incentiv...,"[in, Garut, reward, commitment, to, provide, i...",positif
3027,It's time for us to support to sort out things...,"[It, 's, time, for, us, to, support, to, sort,...",positif


In [33]:
# Memuat stopwords bahasa Inggris dari nltk
stop_words = set(stopwords.words('english'))

# Memuat stopwords tambahan dari file 'stopwords.txt'
additional_stopwords = pd.read_csv('drive/MyDrive/TA/Codingan TA/analisis sentimen /dataset/stopword.txt', header=None)
additional_stopwords = set(additional_stopwords[0].tolist())

# Menggabungkan stopwords dari nltk dan file tambahan
stop_words.update(additional_stopwords)

# Fungsi untuk membersihkan teks dari stopwords
def remove_stopwords(words):
    if isinstance(words, list):
        # Menghapus stopwords dari list kata
        cleaned_words = [word for word in words if word.lower() not in stop_words]
    else:
        # Tokenisasi teks dan menghapus stopwords
        words = words.split()
        cleaned_words = [word for word in words if word.lower() not in stop_words]

    # Menggabungkan kembali kata-kata menjadi satu teks
    return ' '.join(cleaned_words)
# Menghapus stopwords dari kolom teks tertentu, misalnya kolom 'tokenize_tweet'
tweet_label['cleaned_tweet'] = tweet_label['tokenize_tweet'].apply(remove_stopwords)

# Simpan kembali dataset yang sudah dibersihkan
tweet_label.to_csv('drive/MyDrive/TA/Codingan TA/analisis sentimen /dataset/dataset-stopword.csv', index=False)

# Menampilkan beberapa baris pertama dari dataset yang sudah dibersihkan
tweet_label = tweet_label[['tweet_english', 'tokenize_tweet', 'cleaned_tweet','labels']]
tweet_label

,tweet_english,tokenize_tweet,cleaned_tweet,labels
0,Anies urges to form a true leader with the peo...,"[Anies, urges, to, form, a, true, leader, with...","Anies urges form true leader people borders , ...",positif
1,"Mr. Anies is now a father figure for Gen Z, es...","[Mr., Anies, is, now, a, father, figure, for, ...","Mr. Anies father figure Gen , lost father figu...",positif
2,bubble,[bubble],bubble,netral
3,Downloading why whatever we do will produce co...,"[Downloading, why, whatever, we, do, will, pro...",Downloading produce commensurate,netral
4,the support for sir is really beyond expectati...,"[the, support, for, sir, is, really, beyond, e...","support sir expectations happened Indonesia , ...",positif
...,...,...,...,...
3024,"Visiting Banten, Atikoh visited Abuya Muhtadi ...","[Visiting, Banten, ,, Atikoh, visited, Abuya, ...","Visiting Banten , Atikoh visited Abuya Muhtadi...",negatif
3025,Happy coffee,"[Happy, coffee]",Happy coffee,positif
3026,in Garut reward commitment to provide incentiv...,"[in, Garut, reward, commitment, to, provide, i...",Garut reward commitment provide incentives Kor...,positif
3027,It's time for us to support to sort out things...,"[It, 's, time, for, us, to, support, to, sort,...",'s time support sort,positif


Stemming

In [34]:
tweet_lab = tweet_label[['cleaned_tweet', 'labels']]
tweet_lab.head()

,cleaned_tweet,labels
0,"Anies urges form true leader people borders , ...",positif
1,"Mr. Anies father figure Gen , lost father figu...",positif
2,bubble,netral
3,Downloading produce commensurate,netral
4,"support sir expectations happened Indonesia , ...",positif


In [35]:
def clean_text(text):
    return text.replace('.', '').replace(',', '').replace('?', '').replace('!', '').replace('"', '').replace('“', '').replace('”', '').replace('‘', '').replace("'",'').replace("-",'')

# Apply the function to the 'cleaned_tweet' column
tweet_lab['cleaned_tweet'] = tweet_lab['cleaned_tweet'].apply(clean_text)

<ipython-input-35-32e8a89b4159>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweet_lab['cleaned_tweet'] = tweet_lab['cleaned_tweet'].apply(clean_text)


In [36]:
tweet_lab.head()

,cleaned_tweet,labels
0,Anies urges form true leader people borders G...,positif
1,Mr Anies father figure Gen lost father figure...,positif
2,bubble,netral
3,Downloading produce commensurate,netral
4,support sir expectations happened Indonesia a...,positif


In [37]:
# Inisialisasi stemmer
stemmer = PorterStemmer()

# Daftar kata-kata yang tidak akan di-stem
exclude_words = {'anies', 'dont', 'urgent'}

# Fungsi untuk melakukan stemming dengan pengecualian kata tertentu
def stem(text):
    # Mengonversi input ke string (untuk menangani NaN atau tipe data lain)
    text = str(text)
    # Memisahkan kata-kata
    words = text.split()
    # Melakukan stemming pada setiap kata, kecuali yang ada dalam exclude_words
    stemmed_words = [stemmer.stem(word) if word.lower() not in exclude_words else word for word in words]
    # Menggabungkan kata-kata yang sudah di-stem
    return ' '.join(stemmed_words)

# Mengonversi semua nilai di 'cleaned_tweet' ke string dan menangani NaN
tweet_lab['cleaned_tweet'] = tweet_lab['cleaned_tweet'].astype(str).fillna('')

# Menerapkan fungsi stemming pada kolom 'cleaned_tweet'
tweet_lab['stemming_tweet'] = tweet_lab['cleaned_tweet'].apply(stem)

# Menampilkan DataFrame dengan kolom baru hasil stemming
tweet_lab.head(20)

<ipython-input-37-411f700997b9>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweet_lab['cleaned_tweet'] = tweet_lab['cleaned_tweet'].astype(str).fillna('')
<ipython-input-37-411f700997b9>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweet_lab['stemming_tweet'] = tweet_lab['cleaned_tweet'].apply(stem)


,cleaned_tweet,labels,stemming_tweet
0,Anies urges form true leader people borders G...,positif,Anies urg form true leader peopl border god mr...
1,Mr Anies father figure Gen lost father figure...,positif,mr Anies father figur gen lost father figur ab...
2,bubble,netral,bubbl
3,Downloading produce commensurate,netral,download produc commensur
4,support sir expectations happened Indonesia a...,positif,support sir expect happen indonesia averag per...
5,Anies Bubble Trending,netral,Anies bubbl trend
6,heartrending scream ojol driver,negatif,heartrend scream ojol driver
7,rise intelligent people choose,positif,rise intellig peopl choos
8,hope efforts carry special sir prayers pious...,positif,hope effort carri special sir prayer piou peop...
9,s cool amp,positif,s cool amp


In [38]:
tweet_lab.duplicated().sum()

19

In [39]:
tweet_lab = tweet_lab.drop_duplicates()
tweet_lab.duplicated().sum()

0

In [40]:
tweet_lab.isnull().sum()

cleaned_tweet     0
labels            0
stemming_tweet    0
dtype: int64

In [41]:
new = tweet_lab[['stemming_tweet', 'labels']].rename(columns={'stemming_tweet': 'clean_tweet)', 'labels': 'labels'})
new.to_csv('drive/MyDrive/TA/Codingan TA/analisis sentimen /dataset/stemming_tweet.csv', index=False)